# 브런치 사용자를 위한 작가 및 글 추천
## 1팀 : AI Love You

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc # rc == run configure(configuration file)

%matplotlib inline
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf", size=10).get_name()
# rc('font', family=font_name, size=12)

# font_path = 'NanumGothic.ttf'
# font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [4]:
directory = '/Users/jisoo/python_MLDL/Mid_Project/mid_ailoveu_temp/brunch_data/'

## 1. Data Read

### 0. metadata

In [5]:
metadata = pd.read_json('metadata.json', lines=True)
metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782
1,12081,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,,1463092749000,81,@kohwang56_81
2,0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000,4,@hannahajink_4
3,16315,@bryceandjuli,싫다,"[감정, 마음, 위로]",https://brunch.co.kr/@bryceandjuli/88,,1491055161000,88,@bryceandjuli_88
4,29363,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000,34,@mijeongpark_34


### a. user_keyword.xlsx

In [6]:
user = pd.read_excel('user_keyword.xlsx')

In [7]:
user.shape

(100, 2)

In [8]:
user.head()

,user_id,keyword_list
0,#00001ba6ca8d87d2fc34d626ba9cfe6f,"['사랑', '성공', '인생', '브런치', '작가', '글쓰기', '해외취업',..."
1,#0000e87158c1426d6ffb72cebac6cb64,"['브런치', '가해자', '여성혐오', '페미니스트']"
2,#0000eea6d339abfd02ed590bc451fc63,"['인간관계', '해외생활', '회사']"
3,#0000fdba8f35c76eacab74c5c6bc7f1a,"['도쿄', '여행', '그림에세이', '도쿄', '여행', '그림에세이', '운동..."
4,#000127ad0f1981cae1292efdb228f0e9,"['사랑', '결혼', '결혼생활', '사랑', '결혼', '결혼생활', '연애',..."


### b. writer.xlsx

In [9]:
writer_4000 = pd.read_csv('writer_4000most320_keyword_keywords.csv', index_col=0)

In [10]:
writer_4000.shape

(4000, 5)

In [11]:
writer_4000.head()

,writer_id,page_num,keyword_list,keyword_list_20,keywords
0,@bookfit,4106,"['성공', '행복', '인생']","['성공', '행복', '인생', '부동산', '여행', '창업', '사랑', '생...","['창업', '비즈니스', '문제', '경제', '살림살이', '경제학', '미세먼..."
1,@wikitree,3192,"['영화', '뉴스', '연합뉴스']","['영화', '뉴스', '연합뉴스', '경찰', '다이어트', '사진', '여행',...","['지휘관', '장교', '청계천', '서울', 'IT', '번호', '중소기업',..."
2,@jordan777,2797,"['중국', '미국', '트럼프']","['중국', '미국', '트럼프', '부동산', '주식', '투자', '구글', '...","['석유에너지', '베네수엘라', '경제', '일자리', '청년실업', '저출산',..."
3,@hitchwill,2115,"['국내여행', '영화', '여행']","['국내여행', '영화', '여행', '여행정보', '음식', '사랑', '소설',...","['국내여행', '한옥', '청양', '여행', '가을', '코스모스', '감귤농장..."
4,@tenbody,1759,"['다이어트', '운동', '건강']","['다이어트', '운동', '건강', '스트레칭', '근육', '뱃살', '요가',...","['운동', '근육', '다이어트', '운동', '뱃살', '다이어트', '운동',..."


### c. users.json

In [12]:
users_follow = pd.read_json('users.json', lines=True)

In [13]:
# following_list가 있는 유저
following = users_follow[users_follow['following_list'].str.len() !=0 ]

In [14]:
following = following[['id', 'following_list']]
following

,id,following_list
0,#901985d8bc4c481805c4a4f911814c4a,"[@perytail, @brunch]"
1,#1fd89e9dcfa64b45020d9eaca54e0eed,"[@holidaymemories, @wadiz, @sciforus, @dailydu..."
2,#1d94baaea71a831e1f33e1c6bd126ed5,"[@commerceguy, @sunsutu, @kakao-it, @joohoonja..."
3,#04641c01892b12dc018b1410e4928c0d,"[@amberjeon48, @forsy20, @nemotokki, @hawann, ..."
4,#65bcaff862aadff877e461f54187ab62,"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant..."
...,...,...
310752,#6b7a58c1002fe2763bf4db6e223769b1,"[@angiesongc9sx, @merryseo, @psychiatricnews, ..."
310753,#2863e47d50f1640df6dac10b7bad94fb,"[@login002, @kkonal, @leeraha, @tobeme, @sohyu..."
310754,#4341a155d1966e5618e310c45386aea4,"[@simplelife-1p, @mint5051, @thecapitalist, @s..."
310755,#0d70f397a78d2ef638f812592fa8e6ba,"[@cometseeker, @bijou, @suhanjang, @brunch]"


In [15]:
writer_4000

,writer_id,page_num,keyword_list,keyword_list_20,keywords
0,@bookfit,4106,"['성공', '행복', '인생']","['성공', '행복', '인생', '부동산', '여행', '창업', '사랑', '생...","['창업', '비즈니스', '문제', '경제', '살림살이', '경제학', '미세먼..."
1,@wikitree,3192,"['영화', '뉴스', '연합뉴스']","['영화', '뉴스', '연합뉴스', '경찰', '다이어트', '사진', '여행',...","['지휘관', '장교', '청계천', '서울', 'IT', '번호', '중소기업',..."
2,@jordan777,2797,"['중국', '미국', '트럼프']","['중국', '미국', '트럼프', '부동산', '주식', '투자', '구글', '...","['석유에너지', '베네수엘라', '경제', '일자리', '청년실업', '저출산',..."
3,@hitchwill,2115,"['국내여행', '영화', '여행']","['국내여행', '영화', '여행', '여행정보', '음식', '사랑', '소설',...","['국내여행', '한옥', '청양', '여행', '가을', '코스모스', '감귤농장..."
4,@tenbody,1759,"['다이어트', '운동', '건강']","['다이어트', '운동', '건강', '스트레칭', '근육', '뱃살', '요가',...","['운동', '근육', '다이어트', '운동', '뱃살', '다이어트', '운동',..."
...,...,...,...,...,...
3995,@starshines,39,"['영화', '사랑', '전쟁']","['영화', '사랑', '전쟁', '범죄영화', '픽사', '리뷰', '한국영화',...","['영화', '저니스엔드', '전쟁', '영화', '하정우', '민주주의', '히트..."
3996,@jeolmiing,39,"['독서', '독서모임', '인터뷰']","['독서', '독서모임', '인터뷰', '사랑', '모임', '행복', '생각', ...","['독서', '일상', '인터뷰', '후기', '독서모임', '가이드', '인터뷰'..."
3997,@prague,39,"['프라하', '체코', '유럽여행']","['프라하', '체코', '유럽여행', '여행', '체코여행', '까를교', '유럽...","['프라하', '유럽여행', '체코', '유럽여행', '프라하', '체코', '프라..."
3998,@tilltue,39,"['IT', 'iOS', '개발']","['IT', 'iOS', '개발', '이벤트', '테스트', 'event', '프로...","['테스트코드', '테스트', 'IT', 'IT', 'iOS', 'IT', '테스트..."


In [16]:
def get_user_following_author_keywords(id):
    temp = []
    follow_list = list(following.loc[following['id'] == id, 'following_list'])[0]
    for author in follow_list:
        temp.append(list(writer_4000.loc[writer_4000['writer_id'] == author, 'keywords'])[0])
    result = temp[0]
    for i in range(1,len(temp)):
        result += temp[i]
    return result

In [17]:
get_user_following_author_keywords('#901985d8bc4c481805c4a4f911814c4a')

"['그림일기', '부산외대컴퓨터수리', '현진컴퓨터', '모험', '희극', '비극', '로건', '울버린', '엑스맨', 'Instagram', '무책임', '페리', '에너지', '거절', '실패', '오라', '당신', '하늘', '슬픔', '생각', '당신', '비가', '당신', '본격광고만화', '토스터', '기억', '햇살', '바닥', '초록', '당신', '제주', '협재바다', '바다', '꼬옥', '그늘', '평생', '햇살', '시간', '새해', '세상', '항해', '햇살', '바다', '겨울', '당신', '행복', '매일', '로켓', '행복', '잠금화면', '온도', '화면', '웃음', '비결', '기분', '꽃길', '걸음', '당신', '페리', '제주', '시작', '시간', '페리', '제주동쪽', '제주', '풍림다방', '제주', '협재', '미니', '금빛', '순간', '당신', '조각', '욕심', '마음', '유리', '당신', '생각', '페리', '이모티콘', '자가증식', '영화', '라라랜드', '공중', 'BMW', '페리', '메일', '손짓', '바닥', '점점', '메리크리스마스', '크리스마스', '당신', '안녕', '당신', '다음', '페리', '여행', '출발', '순간', '페리', '페리', '부탁', '올해', '버프툰', '웹툰', '웹툰', '공유', '버프', '버프툰', '하루', '일상', '웹툰', '공유', '버프툰', '웹툰', '거리', '웹툰', '버프툰', '구독', '텀블벅', '시간', '버프툰', '일상', '구원', '아이폰', '겨울', '감정가게', '페리', '감정가게', '페리', '당신', '메리크리스마스', '배경화면', '잠금화면', '잠금화면', '감정가게', '페리', '잠금화면', '아이폰', '감정가게', '페리', '안녕', '감정가게', '아이폰', '잠금화면', '감정가게', '텀블벅', '펀딩', '시간', 'love', '페

## unix 시간 변환

In [18]:
metadata['upload_time'] = pd.to_datetime(metadata['reg_ts']/1000, unit='s')

## 시간기준으로 metadata 나누기
### 2019 이전 / 2019 이후

In [19]:
metadata['upload_year'] = metadata['upload_time'].dt.year

In [20]:
before_metadata = metadata[metadata['upload_year'] < 2019]
before_metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id,upload_time,upload_year
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782,2016-09-27 02:47:07,2016
1,12081,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,,1463092749000,81,@kohwang56_81,2016-05-12 22:39:09,2016
2,0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000,4,@hannahajink_4,2015-11-20 05:28:07,2015
3,16315,@bryceandjuli,싫다,"[감정, 마음, 위로]",https://brunch.co.kr/@bryceandjuli/88,,1491055161000,88,@bryceandjuli_88,2017-04-01 13:59:21,2017
4,29363,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000,34,@mijeongpark_34,2018-04-09 16:55:42,2018


In [21]:
after_metadata = metadata[metadata['upload_year'] >= 2019]
after_metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id,upload_time,upload_year
10,39727,@hukho,같은 물건의 다른 삶,"[패션, 에세이, 경제]",https://brunch.co.kr/@hukho/247,MD의 꽃점,1551798000000,247,@hukho_247,2019-03-05 15:00:00,2019
14,40763,@seesawyou,여행 단상 1,"[단상, 여행, 생각]",https://brunch.co.kr/@seesawyou/130,(6) 나는 왜 떠나왔는가 # 2,1547185083000,130,@seesawyou_130,2019-01-11 05:38:03,2019
19,39865,@haninorway19,엄마도 취향이 있다,"[엄마, 취향, 효도]",https://brunch.co.kr/@haninorway19/103,엄마가 좋아하고 싫어하는 걸 알게 된 기쁜 순간,1553232472000,103,@haninorway19_103,2019-03-22 05:27:52,2019
21,40763,@seesawyou,"세계여행, 막상 해보니 어때? 행복해? / 체코 프라하","[세계여행, 여행, 행복]",https://brunch.co.kr/@seesawyou/133,(7) 행복에 대하여 # 1,1547333801000,133,@seesawyou_133,2019-01-12 22:56:41,2019
25,42456,@jerad,‘팀워크’보다 앞서는 ‘개인기’는 없다,"[팀워크, 성과, 개인기]",https://brunch.co.kr/@jerad/200,,1551366000000,200,@jerad_200,2019-02-28 15:00:00,2019


## 2. 유사도 분석

## TF-IDF

### 독자가 읽은 글 기반

In [22]:
# 유사도 분석에 필요한 패키지를 불러온다

from sklearn.feature_extraction.text import TfidfVectorizer #, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# writer_4000 = pd.read_csv(directory + 'writer_4000most3_keyword_keywords.csv', index_col=0)
# 작가 데이터 vectorization
writer_key = []

for i in range(4000):
    writer_key.append(' '.join(eval(writer_4000['keywords'][i])))
    
corpus = writer_key
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).todense()
pd.DataFrame(X)


def reader_read(i):    
    # 독자데이터 vectorization
    read_key = user['keyword_list']
    read_key[i] = eval(read_key[i])
    read1 = ' '.join(read_key[i])
    reader = vectorizer.transform([read1]).todense()
    
    # Cosine similarity 계산
    cosine_similarity(reader, X)
    result = cosine_similarity(reader, X)
    
    # 결과 값 도출
    global writer_4000
    writer_4000['read_sim'] = result[0]
    writer_4000.sort_values(by='read_sim', ascending=False)
    writer_4000 = writer_4000.sort_values(by='read_sim', ascending=False)
    writer_top3 = writer_4000.head(3)
    writer_results = writer_top3['writer_id'].tolist()
    print("%s님께 추천드리는 작가는 %s, %s, %s 입니다."%(user['user_id'][i], writer_results[0], writer_results[1], writer_results[2]))


In [24]:
reader_read(1)

MemoryError: Unable to allocate 2.24 GiB for an array with shape (4000, 75085) and data type float64

### 독자가 구독한 작가 기반 계산

In [ ]:
# def get_user_following_author_keywords(id):
#     temp = []
#     follow_list = list(following.loc[following['id'] == id, 'following_list'])[0]
#     for author in follow_list:
#         temp.append(list(writer_4000.loc[writer_4000['writer_id'] == author, 'keywords'])[0])
#     result = temp[0]
#     for i in range(1,len(temp)):
#         result += temp[i]
#     return result

In [ ]:
def get_user_following_author_keywords(id):
    temp = []
    follow_list = list(following.loc[following['id'] == id, 'following_list'])[0]
    for author in follow_list:
        append_list = writer_4000.loc[writer_4000['writer_id'] == author, 'keywords']
        if len(append_list) > 0:
            append_list = list(append_list)[0]
            temp.append(append_list)

    result = temp[0]
    for i in range(1,len(temp)):
        result += temp[i]
    return result

In [ ]:
writer_4000

In [ ]:
def reader_follow(i):
    global writer_4000
    
    # 독자데이터 vectorization
    list_t = get_user_following_author_keywords(following['id'][i])
    read_to_list = [l + "]" for l in list_t.split("]")][:-1]
    read_to_list = [eval(l) for l in read_to_list]
    read2 =  ' '.join(read_to_list[0])
    reader2 = vectorizer.transform([read2]).todense()
    
    # Cosine similarity 계산
    result_follow = cosine_similarity(reader2, X)
    
    # 결과 값 도출
    writer_4000['follow_sim'] = result_follow[0]
    writer_4000 = writer_4000.sort_values(by='follow_sim', ascending=False)
    writers_top3 = writer_4000.head(3)
    writers_results = writers_top3['writer_id'].tolist()
    
    return writers_results

In [ ]:
writers_results = reader_follow(1)
writers_results
print("%s님께서 팔로우하시는 작가 기반으로 추천드리는 작가는 %s, %s입니다."%(following['id'][1], writers_results[0], writers_results[1]))

In [ ]:
type(writers_results)

In [ ]:
writers_results

### 추천받은 작가기반 글 추천

In [ ]:
article_df = after_metadata.loc[after_metadata['user_id'].isin(writers_results)]

In [ ]:
article_df.head()

In [ ]:
def recoomd_article(recommend_authors, i):
    global article_df
    user_id = following['id'][i]

    reader = get_user_following_author_keywords(user_id)
    reader = reader[1:-1].replace('\'','').replace(',','')
    
    articles = list(after_metadata.loc[after_metadata['user_id'].isin(recommend_authors)]['keyword_list'])
    
    article_key = []

    for i in range(len(articles)):
        article_key.append(' '.join(eval(str(articles[i]))))
        
    corpus = article_key
    vectorizer = TfidfVectorizer()  # TfidfVectorizer() 객체 변수 생성
    X = vectorizer.fit_transform(corpus).todense()
    
    reader2 = vectorizer.transform([reader]).todense()
    
    pd.DataFrame(X) # np.array or np.matrix => pd.DataFrame
    
    result = cosine_similarity(reader2, X)
    article_df['read_sim'] = result[0]
    article_df = article_df.sort_values(by='read_sim', ascending=False)
    
    return article_df.iloc[0]['title']
    
    
    return reader
    
 


In [ ]:
writers_results

In [ ]:
article = recoomd_article(writers_results, 1)
print("%s님께서 추천 받은 작가 중에서 취향에 맞는 글은 \'%s\' 입니다."%(following['id'][1], article))